<img width="800px" src="../images/img/001.png"></img>
# NOM DU PROJET: **Application mobile d'aide à la conduite  de reconnaissance des panneau de circulation routiere**
# <!-- TITLE --> Etape2- premier modele de convolution 
<!--Auteur : Tchotaneu Ngatcha Giresse (INSTITUTE FRANCOPHONE INTERNATIONAL) -->

# <!-- TITLE --> [GTSRB5] - Convolutions complètes avec l'ensembles des datasets
## Objectifs :
  - Essayer plusieurs solutions.
  - Concevoir un code générique et utilisable par lots.

## Ce que nous allons faire :

Nos principales étapes :
 - Essayer n modèles avec n jeux de données


## 1: preparation de l'environement de travail (importation des bibiotheque utiles )

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import h5py
import sys,os,time,json
import random
from IPython.display import display
sys.path.append('..')
VERSION='1.6'
import mesFonctionsPersonnelles.fonctionDeBase as projet

run_dir = './run/GTSRB5'
datasets_dir = projet.init('GTSRB5', run_dir)

<br>**Projet de reconnaissance de forme**

Version              : 2.0.35
Notebook id          : GTSRB5
Run time             : Wednesday 26 October 2022, 17:08:37
Hostname             : DESKTOP-36O6V1H (Windows)
Tensorflow log level : Info + Warning + Error  (=0)
Datasets dir         : C:\Users\TCHOTANEU\LesProjetsGiresse\datasets
Run dir              : ./run/GTSRB5
Update keras cache   : False
tensorflow           : 2.9.1
numpy                : 1.23.2
sklearn              : 1.1.2
matplotlib           : 3.5.3
pandas               : 1.4.3


### 1. - Mise en place des Parametres

In [2]:
enhanced_dir = f'./data'


# ---- uniaue;ent sur les trois dataset
#datasets      = ['set-24x24-L', 'set-24x24-RGB', 'set-48x48-RGB']
#models        = {'v1':'get_model_v1', 'v2':'get_model_v2', 'v3':'get_model_v3'}
#batch_size    = 64
#epochs        = 5
#scale         = 1
#with_datagen  = False
#fit_verbosity = 0

#---- toute les possibilites 
datasets      = ['set-24x24-L', 'set-24x24-RGB', 'set-48x48-L', 'set-48x48-RGB', 'set-24x24-L-LHE', 'set-24x24-RGB-HE', 'set-48x48-L-LHE', 'set-48x48-RGB-HE']
models        = {'v1':'get_model_v1', 'v2':'get_model_v2', 'v3':'get_model_v3'}
batch_size    = 64
epochs        = 20
scale         = 1
with_datagen  = True
fit_verbosity = 1


In [3]:
projet.override('enhanced_dir', 'datasets', 'models', 'batch_size', 'epochs', 'scale', 'with_datagen', 'fit_verbosity')

## Étape 2 - Démarrage

In [4]:
random.seed(time.time())

# ---- Where I am ?
dateDuJour    = time.strftime("%A %d %B %Y - %Hh%Mm%Ss")
here   = os.getcwd()
tag_id = '{:06}'.format(random.randint(0,99999))

# ---- Who I am ?
oar_id   = os.getenv("OAR_JOB_ID",  "??")
slurm_id = os.getenv("SLURM_JOBID", "??")

print('Full Convolutions Notebook :')
print(" la version de notre modele    :", VERSION  )
print("Aujourd'hui est le   : ", dateDuJour     )
print('Tag id             : ', tag_id   )
print("le repectoire de travail  : ", here     )
print("le repectoire de sortir :", run_dir  )
print("pour visualiser avec tensorboard    :", f'--logdir {run_dir}')

Full Convolutions Notebook :
 la version de notre modele    : 1.6
Aujourd'hui est le   :  Wednesday 26 October 2022 - 17h08m58s
Tag id             :  052815
le repectoire de travail  :  C:\Users\TCHOTANEU\LesProjetsGiresse\Projets\GTSRB
le repectoire de sortir : ./run/GTSRB5
pour visualiser avec tensorboard    : --logdir ./run/GTSRB5


## Étape 3 - Chargement du jeu de données

In [5]:
def read_dataset(enhanced_dir, dataset_name):
    '''Reads h5 dataset from dataset_dir
    Args:
        dataset_dir : datasets dir
        name        : dataset name, without .h5
    Returns:    x_train,y_train,x_test,y_test data'''
    # ---- Read dataset
    filename = f'{enhanced_dir}/{dataset_name}.h5'
    size     = os.path.getsize(filename)/(1024*1024)

    with  h5py.File(filename,'r') as f:
        x_train = f['x_train'][:]
        y_train = f['y_train'][:]
        x_test  = f['x_test'][:]
        y_test  = f['y_test'][:]

    # ---- melanger les données
    x_train,y_train=pwk.shuffle_np_dataset(x_train,y_train)

    # ---
    return x_train,y_train,x_test,y_test,size

## Étape 4 - Collecte des modèles

In [6]:

# model simple
#
def get_model_v1(lx,ly,lz):
    
    model = keras.models.Sequential()
    
    model.add( keras.layers.Conv2D(96, (3,3), activation='relu', input_shape=(lx,ly,lz)))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Conv2D(192, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D((2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Flatten()) 
    model.add( keras.layers.Dense(1500, activation='relu'))
    model.add( keras.layers.Dropout(0.5))

    model.add( keras.layers.Dense(43, activation='softmax'))
    return model
    
# deux modeles complexes
#
def get_model_v2(lx,ly,lz):
    model = keras.models.Sequential()

    model.add( keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(lx,ly,lz), activation='relu'))
    model.add( keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add( keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Conv2D(256, (3, 3), padding='same',activation='relu'))
    model.add( keras.layers.Conv2D(256, (3, 3), activation='relu'))
    model.add( keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add( keras.layers.Dropout(0.2))

    model.add( keras.layers.Flatten())
    model.add( keras.layers.Dense(512, activation='relu'))
    model.add( keras.layers.Dropout(0.5))
    model.add( keras.layers.Dense(43, activation='softmax'))
    return model

def get_model_v3(lx,ly,lz):
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (5, 5), padding='same',  activation='relu', input_shape=(lx,ly,lz)))
    model.add(tf.keras.layers.BatchNormalization(axis=-1))      
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same',  activation='relu'))
    model.add(tf.keras.layers.BatchNormalization(axis=-1))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization(axis=-1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))

    model.add(tf.keras.layers.Dense(43, activation='softmax'))
    return model

## Étape 5 - Ensembles de données multiples, modèles multiples

In [7]:
def multi_run(enhanced_dir, datasets, models, datagen=None,
              scale=1, batch_size=64, epochs=20, 
              fit_verbosity=0, tag_id='last'):
      
    # -
    #
    os.makedirs(f'{run_dir}/logs_{tag_id}',   mode=0o750, exist_ok=True)
    os.makedirs(f'{run_dir}/models_{tag_id}', mode=0o750, exist_ok=True)
    
    # ---- 
    #
    output={}
    output['Dataset'] = []
    output['Size']    = []
    for m in models:
        output[m+'_Accuracy'] = []
        output[m+'_Duration'] = []

    # ---- 
    #
    for d_name in datasets:
        print("\nDataset : ",d_name)

        # ---- lire le  dataset
        x_train,y_train,x_test,y_test, d_size = read_dataset(enhanced_dir, d_name)
        output['Dataset'].append(d_name)
        output['Size'].append(d_size)
        
        # ---- 
        x_train,y_train,x_test,y_test = pwk.rescale_dataset(x_train,y_train,x_test,y_test, scale=scale)
        
        # ---- 
        (n,lx,ly,lz) = x_train.shape

        # ---- pour chaque modele
        for m_name,m_function in models.items():
            print("    Run model {}  : ".format(m_name), end='')
            # ---- choisir le modele
            try:
                # ---- 
                m_function=globals()[m_function]
                model=m_function(lx,ly,lz)
                # ---- compiler le modele 
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                # ---- Callbacks 
                log_dir = f'{run_dir}/logs_{tag_id}/tb_{d_name}_{m_name}'
                tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
                # ---- Callbacks bestmodel
                save_dir = f'{run_dir}/models_{tag_id}/model_{d_name}_{m_name}.h5'
                bestmodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, monitor='accuracy', save_best_only=True)
                # ---- Train
                start_time = time.time()
                if datagen==None:
                    # ---- sans la data augmentation  --------------------------------------
                    history = model.fit(x_train, y_train,
                                        batch_size      = batch_size,
                                        epochs          = epochs,
                                        verbose         = fit_verbosity,
                                        validation_data = (x_test, y_test),
                                        callbacks       = [tensorboard_callback, bestmodel_callback])
                else:
                    # ---- avec la data augmentation  ----------------------------------------
                    datagen.fit(x_train)
                    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                                        steps_per_epoch = int(len(x_train)/batch_size),
                                        epochs          = epochs,
                                        verbose         = fit_verbosity,
                                        validation_data = (x_test, y_test),
                                        callbacks       = [tensorboard_callback, bestmodel_callback])
                    
                # ---- Resultats
                end_time = time.time()
                duration = end_time-start_time
                accuracy = max(history.history["val_accuracy"])*100
                #
                output[m_name+'_Accuracy'].append(accuracy)
                output[m_name+'_Duration'].append(duration)
                print(f"Accuracy={accuracy: 7.2f}    Duration={duration: 7.2f}")
            except:
                raise
                output[m_name+'_Accuracy'].append('0')
                output[m_name+'_Duration'].append('999')
                print('-')
    return output

## Étape 6 - Exécution

In [8]:
projet.chrono_start()

print('\n---- Run','-'*50)

#
if with_datagen :
    datagen = keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
                                                           featurewise_std_normalization=False,
                                                           width_shift_range=0.1,
                                                           height_shift_range=0.1,
                                                           zoom_range=0.2,
                                                           shear_range=0.1,
                                                           rotation_range=10.)
else:
    datagen=None
    
# ---- execution
#
output = multi_run(enhanced_dir,
                   datasets, 
                   models,
                   datagen       = datagen,
                   scale         = scale,
                   batch_size    = batch_size,
                   epochs        = epochs,
                   fit_verbosity = fit_verbosity,
                   tag_id        = tag_id)

# ---- sauvegarte le rapport
#
report={}
report['output']=output
report['description'] = f'scale={scale} batch_size={batch_size} epochs={epochs} data_aug={with_datagen}'

report_name=f'{run_dir}/report_{tag_id}.json'

with open(report_name, 'w') as file:
    json.dump(report, file, indent=4)

print('\nReport saved as ',report_name)

projet.chrono_show()
print('-'*59)



---- Run --------------------------------------------------

Dataset :  set-24x24-L
    Run model v1  : Epoch 1/20
612/612 [==============================] - 63s 103ms/step - loss: 2.1439 - accuracy: 0.3995 - val_loss: 0.8256 - val_accuracy: 0.7647
Epoch 2/20
612/612 [==============================] - 62s 102ms/step - loss: 0.9250 - accuracy: 0.7189 - val_loss: 0.4790 - val_accuracy: 0.8694
Epoch 3/20
612/612 [==============================] - 62s 101ms/step - loss: 0.6129 - accuracy: 0.8094 - val_loss: 0.3236 - val_accuracy: 0.9137
Epoch 4/20
612/612 [==============================] - 63s 102ms/step - loss: 0.4694 - accuracy: 0.8545 - val_loss: 0.2741 - val_accuracy: 0.9272
Epoch 5/20
612/612 [==============================] - 61s 99ms/step - loss: 0.3749 - accuracy: 0.8817 - val_loss: 0.2473 - val_accuracy: 0.9325
Epoch 6/20
612/612 [==============================] - 63s 103ms/step - loss: 0.3188 - accuracy: 0.8995 - val_loss: 0.2104 - val_accuracy: 0.9410
Epoch 7/20
612/612 [======

## Étape 7 -analysons les resultats 

In [9]:
projet.end()

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'NoneType'